In [42]:
import pandas as pd
import numpy as np
import re
import torch
# from torch._C import *
import torch.nn as nn
import random
import json
import os
import string
import torch.nn.functional as F
import tensorflow_datasets.public_api as tfds
import gensim
from gensim.models import Word2Vec
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [44]:
file = open('/content/drive/My Drive/BiGPatentSample.txt',mode='r')
 
# read all lines at once
my_data = file.read()
 
# close the file
file.close()






In [45]:
stop_file = open("/content/drive/My Drive/NLTK's list of english stopwords",mode='r')
stop_words = stop_file.read()
stop_file.close()
stop_words = stop_words.split('\n')
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [46]:
my_data = my_data.split('}')

x = ""
for itm in my_data :
  abst =  itm.split("\"abstract\":",1)
  if len(abst)!=1:
    abst = abst[1].split("\"application_number\":")[0]
    # x.append(abst)
    x = x + abst
  full_txt = itm.split("\"description\":")
  if len(full_txt)!=1:
    # x.append(full_txt[1])
    x = x + full_txt[1]

In [47]:
def clean(text):
  text = str(text)
  text = text.lower()
  text = re.sub(r'\'s',r'\tis',text)
  text = re.sub(r'\'ll',r'\twill',text)
  text = re.sub(r'\'m',r'\tam',text)
  text = re.sub(r'\'re',r'\tare',text)
  text = re.sub(r'\'d',r'\twould',text)
  text = re.sub(r'n\'t',r'\tnot',text)
  # print("1***",text)
  for itm in stop_words:
    text = re.sub(' '+itm +' ',' ',text)
    text = re.sub('"'+ itm + ' ',' ', text)
  
  # print("2***", text)
  text = re.sub('fig[0-9]+',' ',text)
  text = re.sub('[^a-zA-Z0-9]',' ',text) 
  text = re.sub('\w{1,}[0-9]+', ' ', text)
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
  text = re.sub('^a$','', text)
  return text

In [48]:
common_texts = clean(x)

In [53]:
# model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)
model1 = gensim.models.Word2Vec(common_texts, min_count = 1,   size = 300, window = 12) 
model1.save("word2vec.model")